# Intel® Open Image Denoise - CPU and GPU

## Module Overview
Intel Open Image Denoise is an open source library of high-performance, high-quality denoising filters for images rendered with ray tracing. Intel Open Image Denoise is part of the Intel® oneAPI Rendering Toolkit and is released under the permissive Apache 2.0 license.

## Learning Objectives

- **Understanding Intel® Open Image Denoise (OIDN)**: Summarize the purpose and functionality of Intel® Open Image Denoise, including its role in reducing rendering times in ray tracing applications and its support for various rendering solutions.

- **Exploring OIDN API**: Inspect the API presented in the code walkthrough and recall the setup, filter creation, and filter parameters involved in using Intel® Open Image Denoise.

- **Recognizing Supported Platforms**: Identify the various CPUs and GPUs supported by Intel® Open Image Denoise, including architectures and vendors, and understand its efficiency across different hardware configurations.

- **Understanding OIDN Features**: Learn about the features provided by Intel® Open Image Denoise, including its C99 API, object-oriented design, and thread safety, and comprehend the impact of setting parameters and object commits.

- **Building and Executing OIDN**: Learn how to build and execute the oidnDenoise program targeting both CPU and GPU devices using the provided scripts and command-line options.

- **Exploring Optional Inputs**: Understand how to introduce optional input files such as albedo and normal maps for denoising using OIDN, and recognize their significance in enhancing denoising quality.

- **Visualizing Denoised Images**: Advise to use ImageMagick to visualize the denoised images produced by OIDN, facilitating the understanding of denoising results.


***

### 1. Intel&reg; Open Image Denoise Overview

Intel Open Image Denoise is an open source library of high-performance, high-quality denoising filters for images rendered with ray tracing. Intel Open Image Denoise is part of the Intel® Rendering Toolkit and is released under the permissive Apache 2.0 license.

The purpose of Intel Open Image Denoise is to provide an open, high-quality, efficient, and easy-to-use denoising library that allows one to significantly reduce rendering times in ray tracing based rendering applications. It filters out the Monte Carlo noise inherent to stochastic ray tracing methods like path tracing, reducing the amount of necessary samples per pixel by even multiple orders of magnitude (depending on the desired closeness to the ground truth). A simple but flexible C/C++ API ensures that the library can be easily integrated into most existing or new rendering solutions.

At the heart of the Intel Open Image Denoise library is a collection of efficient deep learning based denoising filters, which were trained to handle a wide range of samples per pixel (spp), from 1 spp to almost fully converged. Thus it is suitable for both preview and final-frame rendering. The filters can denoise images either using only the noisy color (beauty) buffer, or, to preserve as much detail as possible, can optionally utilize auxiliary feature buffers as well (e.g. albedo, normal). Such buffers are supported by most renderers as arbitrary output variables (AOVs) or can be usually implemented with little effort.

Although the library ships with a set of pre-trained filter models, it is not mandatory to use these. To optimize a filter for a specific renderer, sample count, content type, scene, etc., it is possible to train the model using the included training toolkit and user-provided image datasets.

***
### 2. Supported Platforms by Open Image Denoise

Intel Open Image Denoise supports a wide variety of CPUs and GPUs from different vendors:

- Intel® 64 architecture compatible CPUs (with at least SSE4.1)

- ARM64 (AArch64) architecture CPUs (e.g. Apple silicon CPUs)

- Intel Xe architecture dedicated and integrated GPUs, including Intel® Arc™ A-Series Graphics, Intel® Data Center GPU Flex Series, Intel® Data Center GPU Max Series, Intel® Iris® Xe Graphics, Intel® Core™ Ultra Processors with Intel® Arc™ Graphics, 11th-14th Gen Intel® Core™ processor graphics, and related Intel Pentium® and Celeron® processors (Xe-LP, Xe-LPG, Xe-HPG, and Xe-HPC microarchitectures)

- NVIDIA GPUs with Volta, Turing, Ampere, Ada Lovelace, and Hopper architectures

- AMD GPUs with RDNA2 (Navi 21 only) and RDNA3 (Navi 3x) architectures

- Apple silicon GPUs (M1 and newer)

It runs on most machines ranging from laptops to workstations and compute nodes in HPC systems. It is efficient enough to be suitable not only for offline rendering, but, depending on the hardware used, also for interactive or even real-time ray tracing.

Intel Open Image Denoise exploits modern instruction sets like SSE4, AVX2, AVX-512, and NEON on CPUs, Intel® Xe Matrix Extensions (Intel® XMX) on Intel GPUs, and tensor cores on NVIDIA GPUs to achieve high denoising performance.

***
### 3. Open Image Denoise Features

Open Image Denoise provides a C99 API (also compatible with C++) and a C++11 wrapper API as well. For simplicity, this document mostly refers to the C99 version of the API.

The API is designed in an object-oriented manner, e.g. it contains device objects (OIDNDevice type), buffer objects (OIDNBuffer type), and filter objects (OIDNFilter type). All objects are reference-counted, and handles can be released by calling the appropriate release function (e.g. oidnReleaseDevice) or retained by incrementing the reference count (e.g. oidnRetainDevice).

An important aspect of objects is that setting their parameters do not have an immediate effect (with a few exceptions). Instead, objects with updated parameters have to be committed to a given object. The commit semantic allows for batching up multiple small changes, and specifies exactly when changes to objects will occur.

All API calls are thread-safe, but operations that use the same device will be serialized, so the amount of API calls from different threads should be minimized.

***
### 4. Code Walkthrough (oidnDenoise.cpp)

In [1]:
%%writefile src/apps/oidnDenoise.cpp

// Copyright 2018 Intel Corporation
// SPDX-License-Identifier: Apache-2.0

#include "common/common.h"
#include "common/timer.h"
#include "utils/arg_parser.h"
#include "utils/image_io.h"
#include "utils/device_info.h"
#include <iostream>
#include <fstream>
#include <iomanip>
#include <cassert>
#include <limits>
#include <cmath>
#include <signal.h>
#ifdef VTUNE
#include <ittnotify.h>
#endif

OIDN_NAMESPACE_USING

void printUsage()
{
  std::cout << "Intel(R) Open Image Denoise" << std::endl;
  std::cout << "usage: oidnDenoise [-d/--device [0-9]+|default|cpu|sycl|cuda|hip]" << std::endl
            << "                   [-f/--filter RT|RTLightmap]" << std::endl
            << "                   [--hdr color.pfm] [--ldr color.pfm] [--srgb] [--dir directional.pfm]" << std::endl
            << "                   [--alb albedo.pfm] [--nrm normal.pfm] [--clean_aux]" << std::endl
            << "                   [--is/--input_scale value]" << std::endl
            << "                   [-o/--output output.pfm] [-r/--ref reference_output.pfm]" << std::endl
            << "                   [-t/--type float|half]" << std::endl
            << "                   [-q/--quality default|h|high|b|balanced]" << std::endl
            << "                   [-w/--weights weights.tza]" << std::endl
            << "                   [--threads n] [--affinity 0|1] [--maxmem MB] [--inplace]" << std::endl
            << "                   [-n times_to_run] [-v/--verbose 0-3]" << std::endl
            << "                   [--ld|--list_devices] [-h/--help]" << std::endl;
}

void errorCallback(void* userPtr, Error error, const char* message)
{
  throw std::runtime_error(message);
}

volatile bool isCancelled = false;

void signalHandler(int signal)
{
  isCancelled = true;
}

bool progressCallback(void* userPtr, double n)
{
  if (isCancelled)
  {
    std::cout << std::endl;
    return false;
  }
  std::cout << "\rDenoising " << int(n * 100.) << "%" << std::flush;
  return true;
}

std::vector<char> loadFile(const std::string& filename)
{
  std::ifstream file(filename, std::ios::binary);
  if (file.fail())
    throw std::runtime_error("cannot open file: '" + filename + "'");
  file.seekg(0, file.end);
  const size_t size = file.tellg();
  file.seekg(0, file.beg);
  std::vector<char> buffer(size);
  file.read(buffer.data(), size);
  if (file.fail())
    throw std::runtime_error("error reading from file: '" + filename + "'");
  return buffer;
}

int main(int argc, char* argv[])
{
  DeviceType deviceType = DeviceType::Default;
  PhysicalDeviceRef physicalDevice;
  std::string filterType = "RT";
  std::string colorFilename, albedoFilename, normalFilename;
  std::string outputFilename, refFilename;
  std::string weightsFilename;
  Quality quality = Quality::Default;
  bool hdr = false;
  bool srgb = false;
  bool directional = false;
  float inputScale = std::numeric_limits<float>::quiet_NaN();
  bool cleanAux = false;
  Format dataType = Format::Undefined;
  int numRuns = 1;
  int numThreads = -1;
  int setAffinity = -1;
  int maxMemoryMB = -1;
  bool inplace = false;
  int verbose = -1;

  // Parse the arguments
  if (argc == 1)
  {
    printUsage();
    return 1;
  }

  try
  {
    ArgParser args(argc, argv);
    while (args.hasNext())
    {
      std::string opt = args.getNextOpt();
      if (opt == "d" || opt == "dev" || opt == "device")
      {
        std::string value = args.getNext();
        if (isdigit(value[0]))
          physicalDevice = fromString<int>(value);
        else
          deviceType = fromString<DeviceType>(value);
      }
      else if (opt == "f" || opt == "filter")
        filterType = args.getNextValue();
      else if (opt == "hdr")
      {
        colorFilename = args.getNextValue();
        hdr = true;
      }
      else if (opt == "ldr")
      {
        colorFilename = args.getNextValue();
        hdr = false;
      }
      else if (opt == "srgb")
        srgb = true;
      else if (opt == "dir")
      {
        colorFilename = args.getNextValue();
        directional = true;
      }
      else if (opt == "alb" || opt == "albedo")
        albedoFilename = args.getNextValue();
      else if (opt == "nrm" || opt == "normal")
        normalFilename = args.getNextValue();
      else if (opt == "o" || opt == "out" || opt == "output")
        outputFilename = args.getNextValue();
      else if (opt == "r" || opt == "ref" || opt == "reference")
        refFilename = args.getNextValue();
      else if (opt == "is" || opt == "input_scale" || opt == "input-scale" || opt == "inputScale" || opt == "inputscale")
        inputScale = args.getNextValue<float>();
      else if (opt == "clean_aux" || opt == "clean-aux" || opt == "cleanAux" || opt == "cleanaux")
        cleanAux = true;
      else if (opt == "t" || opt == "type")
      {
        const auto val = toLower(args.getNextValue());
        if (val == "f" || val == "float" || val == "fp32")
          dataType = Format::Float;
        else if (val == "h" || val == "half" || val == "fp16")
          dataType = Format::Half;
        else
          throw std::runtime_error("invalid data type");
      }
      else if (opt == "q" || opt == "quality")
      {
        const auto val = toLower(args.getNextValue());
        if (val == "default")
          quality = Quality::Default;
        else if (val == "h" || val == "high")
          quality = Quality::High;
        else if (val == "b" || val == "balanced")
          quality = Quality::Balanced;
        else
          throw std::runtime_error("invalid filter quality mode");
      }
      else if (opt == "w" || opt == "weights")
        weightsFilename = args.getNextValue();
      else if (opt == "n")
        numRuns = std::max(args.getNextValue<int>(), 1);
      else if (opt == "threads")
        numThreads = args.getNextValue<int>();
      else if (opt == "affinity")
        setAffinity = args.getNextValue<int>();
      else if (opt == "maxmem" || opt == "maxMemoryMB")
        maxMemoryMB = args.getNextValue<int>();
      else if (opt == "inplace")
        inplace = true;
      else if (opt == "v" || opt == "verbose")
        verbose = args.getNextValue<int>();
      else if (opt == "ld" || opt == "list_devices" || opt == "list-devices" || opt == "listDevices" || opt == "listdevices")
        return printPhysicalDevices();
      else if (opt == "h" || opt == "help")
      {
        printUsage();
        return 1;
      }
      else
        throw std::invalid_argument("invalid argument '" + opt + "'");
    }

  #if defined(OIDN_ARCH_X64)
    // Set MXCSR flags
    if (!refFilename.empty())
    {
      // In reference mode we have to disable the FTZ and DAZ flags to get accurate results
      _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_OFF);
      _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_OFF);
    }
    else
    {
      // Enable the FTZ and DAZ flags to maximize performance
      _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
      _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
    }
  #endif

    // Initialize the denoising device
    std::cout << "Initializing device" << std::endl;
    Timer timer;

    DeviceRef device;
    if (physicalDevice)
      device = physicalDevice.newDevice();
    else
      device = newDevice(deviceType);

    const char* errorMessage;
    if (device.getError(errorMessage) != Error::None)
      throw std::runtime_error(errorMessage);
    device.setErrorFunction(errorCallback);

    if (numThreads > 0)
      device.set("numThreads", numThreads);
    if (setAffinity >= 0)
      device.set("setAffinity", bool(setAffinity));
    if (verbose >= 0)
      device.set("verbose", verbose);
    device.commit();

    const double deviceInitTime = timer.query();

    deviceType = device.get<DeviceType>("type");
    const int versionMajor = device.get<int>("versionMajor");
    const int versionMinor = device.get<int>("versionMinor");
    const int versionPatch = device.get<int>("versionPatch");

    std::cout << "  device=" << deviceType
              << ", version=" << versionMajor << "." << versionMinor << "." << versionPatch
              << ", msec=" << (1000. * deviceInitTime) << std::endl;

    // Load the input image
    std::shared_ptr<ImageBuffer> input, ref;
    std::shared_ptr<ImageBuffer> color, albedo, normal;

    std::cout << "Loading input" << std::endl;

    if (!albedoFilename.empty())
      input = albedo = loadImage(device, albedoFilename, 3, false, dataType);

    if (!normalFilename.empty())
      input = normal = loadImage(device, normalFilename, 3, dataType);

    if (!colorFilename.empty())
      input = color = loadImage(device, colorFilename, 3, srgb, dataType);

    if (!input)
      throw std::runtime_error("no input image specified");

    if (!refFilename.empty())
    {
      ref = loadImage(device, refFilename, 3, srgb, dataType);
      if (ref->getDims() != input->getDims())
        throw std::runtime_error("invalid reference output image");
    }

    const int width  = input->getW();
    const int height = input->getH();
    std::cout << "Resolution: " << width << "x" << height << std::endl;

    // Initialize the output image
    std::shared_ptr<ImageBuffer> output;
    if (inplace)
      output = input;
    else
      output = std::make_shared<ImageBuffer>(device, width, height, 3, input->getDataType());

    std::shared_ptr<ImageBuffer> inputCopy;
    if (inplace && numRuns > 1)
      inputCopy = input->clone();

    // Load the filter weights if specified
    std::vector<char> weights;
    if (!weightsFilename.empty())
    {
      std::cout << "Loading filter weights" << std::endl;
      weights = loadFile(weightsFilename);
    }

    // Initialize the denoising filter
    std::cout << "Initializing filter" << std::endl;
    timer.reset();

    FilterRef filter = device.newFilter(filterType.c_str());

    if (color)
      filter.setImage("color", color->getBuffer(), color->getFormat(), color->getW(), color->getH());
    if (albedo)
      filter.setImage("albedo", albedo->getBuffer(), albedo->getFormat(), albedo->getW(), albedo->getH());
    if (normal)
      filter.setImage("normal", normal->getBuffer(), normal->getFormat(), normal->getW(), normal->getH());

    filter.setImage("output", output->getBuffer(), output->getFormat(), output->getW(), output->getH());

    if (filterType == "RT")
    {
      if (hdr)
        filter.set("hdr", true);
      if (srgb)
        filter.set("srgb", true);
    }
    else if (filterType == "RTLightmap")
    {
      if (directional)
        filter.set("directional", true);
    }

    if (std::isfinite(inputScale))
      filter.set("inputScale", inputScale);

    if (cleanAux)
      filter.set("cleanAux", cleanAux);

    if (quality != Quality::Default)
      filter.set("quality", quality);

    if (maxMemoryMB >= 0)
      filter.set("maxMemoryMB", maxMemoryMB);

    if (!weights.empty())
      filter.setData("weights", weights.data(), weights.size());

    const bool showProgress = verbose <= 1;
    if (showProgress)
    {
      filter.setProgressMonitorFunction(progressCallback);
      signal(SIGINT, signalHandler);
    }

    filter.commit();

    const double filterInitTime = timer.query();

    std::cout << "  filter=" << filterType
              << ", msec=" << (1000. * filterInitTime) << std::endl;

    // Denoise the image
    uint32_t prevHash = 0;
    for (int run = 0; run < numRuns; ++run)
    {
      if (inplace && run > 0)
        memcpy(input->getData(), inputCopy->getData(), inputCopy->getByteSize());

      if (!showProgress)
        std::cout << "Denoising" << std::endl;
      timer.reset();

      filter.execute();

      const double denoiseTime = timer.query();

      if (showProgress)
        std::cout << std::endl;
      std::cout << "  msec=" << (1000. * denoiseTime);

      if (numRuns > 1 || verbose >= 2)
      {
        // Compute a hash of the output
        const size_t numBytes = output->getByteSize();
        const uint8_t* outputBytes = static_cast<const uint8_t*>(output->getData());
        uint32_t hash = 0x811c9dc5;
        for (size_t i = 0; i < numBytes; ++i)
        {
          hash ^= outputBytes[i];
          hash *= 0x1000193;
        }
        std::cout << ", hash=" << std::hex << std::setfill('0') << std::setw(8) << hash << std::dec << std::endl;

        if (run > 0 && hash != prevHash)
          throw std::runtime_error("output hash mismatch (non-deterministic output)");
        prevHash = hash;
      }
      else
        std::cout << std::endl;

      if (run == 0 && ref)
      {
        // Verify the output values
        std::cout << "Verifying output" << std::endl;

        const double errorThreshold = (input == normal || directional) ? 0.05 : 0.003;
        size_t numErrors;
        double avgError;
        std::tie(numErrors, avgError) = compareImage(*output, *ref, errorThreshold);

        std::cout << "  values=" << output->getSize()
                  << ", errors=" << numErrors << ", avgerror=" << avgError << std::endl;

        if (numErrors > 0)
        {
          // Save debug images
          std::cout << "Saving debug images" << std::endl;
          saveImage("denoise_in.pfm",  *input,  srgb);
          saveImage("denoise_out.pfm", *output, srgb);
          saveImage("denoise_ref.pfm", *ref,    srgb);

          throw std::runtime_error("output does not match the reference");
        }
      }
    }

    if (showProgress)
    {
      filter.setProgressMonitorFunction(nullptr);
      signal(SIGINT, SIG_DFL);
    }

    if (!outputFilename.empty())
    {
      // Save output image
      std::cout << "Saving output" << std::endl;
      saveImage(outputFilename, *output, srgb);
    }
  }
  catch (const std::exception& e)
  {
    std::cerr << "Error: " << e.what() << std::endl;
    return 1;
  }

  return 0;
}

Overwriting src/apps/oidnDenoise.cpp


***
### 5. Build

In [2]:
! ./build.sh

 
:: WARNING: setvars.sh has already been run. Skipping re-execution.
   To force a re-execution of setvars.sh, use the '--force' option.
   Using '--force' can result in excessive use of your environment variables.
  
usage: source setvars.sh [--force] [--config=file] [--help] [...]
  --force        Force setvars.sh to re-run, doing so may overload environment.
  --config=file  Customize env vars using a setvars.sh configuration file.
  --help         Display this help message and exit.
  ...            Additional args are passed to individual env/vars.sh scripts
                 and should follow this script's arguments.
  
  Some POSIX shells do not accept command-line options. In that case, you can pass
  command-line options via the SETVARS_ARGS environment variable. For example:
  
  $ SETVARS_ARGS="ia32 --config=config.txt" ; export SETVARS_ARGS
  $ . path/to/setvars.sh
  
  The SETVARS_ARGS environment variable is cleared on exiting setvars.sh.
  
-- The C compiler identificati

***
### 6. Explore the oidnDenoise Options

Once the program has been built, we can execute it with the ```--help``` flag to see the options.

In the output of the cell below, we can see that the device to target is introduced as an argument with the ```-d``` flag, which options are ```default```, ```cpu```, ```syc```, ```cuda```, and ```hip```.


In [3]:
output = !./bin/oidnDenoise --help
for l in output:
    print(l)

Intel(R) Open Image Denoise
usage: oidnDenoise [-d/--device [0-9]+|default|cpu|sycl|cuda|hip]
                   [-f/--filter RT|RTLightmap]
                   [--hdr color.pfm] [--ldr color.pfm] [--srgb] [--dir directional.pfm]
                   [--alb albedo.pfm] [--nrm normal.pfm] [--clean_aux]
                   [--is/--input_scale value]
                   [-o/--output output.pfm] [-r/--ref reference_output.pfm]
                   [-t/--type float|half]
                   [-q/--quality default|h|high|b|balanced]
                   [-w/--weights weights.tza]
                   [--threads n] [--affinity 0|1] [--maxmem MB] [--inplace]
                   [-n times_to_run] [-v/--verbose 0-3]
                   [--ld|--list_devices] [-h/--help]


In this tutorial we are going to use ```cpu``` and ```sycl``` to target the CPU and GPU respectively.

***
###  7. Run Targeting CPU

With the following cell, which is equivalent to running the ```./run_CPU.sh``` script, we execute the ```oidnDenoise``` program targeting the CPU using the ```cpu``` flag.

Please note that for now, we are using only one input file, JunkShop_00064spp.hdr.pfm, presented with the ```-hdr``` flag. The inputs for the ```-alb``` flag (for albedo) and for the ```-nrm``` flag (for normal) are optional.

In [4]:
%%bash
./bin/oidnDenoise -d cpu \
-hdr /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.hdr.pfm \
-o JunkShop_denoised_CPU.pfm

Initializing device
  device=CPU, version=2.1.0, msec=350.682
Loading input
Resolution: 2000x1000
Initializing filter
  filter=RT, msec=10.8132
Denoising 100%
  msec=126.526
Saving output


***
### 8. Run Targeting GPU

Similarly, to target the GPU device, can we execute the oidnDenoise program with the ```sycl``` toggle. The cell below is equivalent to running the ```./run_GPU.sh``` script.

In [5]:
%%bash
./bin/oidnDenoise -d sycl \
-hdr /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.hdr.pfm \
-o JunkShop_denoised_GPU.pfm

Initializing device
  device=SYCL, version=2.1.0, msec=257.14
Loading input
Resolution: 2000x1000
Initializing filter
  filter=RT, msec=5.49611
Denoising 100%
  msec=17.6304
Saving output


***
### 9. Run With Albedo and Normal Inputs


With the following cell, we can identify which pfm files located in the common folder of Intel Developer Cloud we can use as input for the ```oidnDenoise``` program. Please note that all these pfm files are pictures with noise that we can denoise using ```oidnDenoise```.

In [6]:
%%bash
ls /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images

JunkShop_00064spp.alb.pfm
JunkShop_00064spp.hdr.pfm
JunkShop_00064spp.hdr.png
JunkShop_00064spp.nrm.pfm
JunkShop_00064spp.nrm.png
JunkShop_00064spp.out.pfm
JunkShop_00064spp.out.png
JunkShop_00064spp_hdr.out.pfm


Where:
* ***.hdr.pfm (color images)
* ***.alb.pfm (albedo images)
* ***.nrm.pfm (normal of surfaces)

The following running script introduces the optional input files for the ```-alb``` and ```-nrm``` flags targeting the CPU.

In [7]:
%%bash
./bin/oidnDenoise -d cpu \
-hdr /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.hdr.pfm \
-alb /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.alb.pfm \
-nrm /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.nrm.pfm \
-o JunkShop_denoised_CPU.pfm

Initializing device
  device=CPU, version=2.1.0, msec=327.065
Loading input
Resolution: 2000x1000
Initializing filter
  filter=RT, msec=12.2394
Denoising 100%
  msec=131.619
Saving output


And this is the running script that introduces the optional input files for the ```-alb``` and ```-nrm``` flags targeting the GPU.

In [8]:
%%bash
./bin/oidnDenoise -d sycl \
-hdr /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.hdr.pfm \
-alb /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.alb.pfm \
-nrm /home/common/data/Big_Data/IRTK_BD/OIDN_BD/images/JunkShop_00064spp.nrm.pfm \
-o JunkShop_denoised_GPU.pfm

Initializing device
  device=SYCL, version=2.1.0, msec=241.446
Loading input
Resolution: 2000x1000
Initializing filter
  filter=RT, msec=5.70927
Denoising 100%
  msec=20.0766
Saving output


To display the denoised images we recommend to download them and use ImageMagick to visualize them.

***

## Summary
In this module you learned:

- About Intel® Open Image Denoise (OIDN).
- About the OIDN API.
- The architecture supported by OIDN.
- The OIDN features.
- How to build and execute OIDN.
- How to introduce optional inputs.
- How to visualizing Denoised Images.

## Resources
* [github.com/OpenImageDenoise](https://github.com/OpenImageDenoise/oidn)
* [openimagedenoise](https://www.openimagedenoise.org/)
* [rendering-toolkit](https://www.intel.com/content/www/us/en/developer/tools/oneapi/rendering-toolkit.html)


***